In [10]:
# Import Libraries and Data
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
import scorecardpy as sc

df = pd.read_csv("/Users/wenwei/Documents/Sku/y4s1/bt4222/project/twitter_human_bots_dataset.csv")
df.head()

,Unnamed: 0,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,0,2016-10-15 21:32:11,False,False,"Blame @xaiax, Inspired by @MakingInvisible, us...",4,1589,4,False,787405734442958848,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/7874121826...,best_in_dumbest,11041,False,7.870,1403,bot
1,1,2016-11-09 05:01:30,False,False,Photographing the American West since 1980. I ...,536,860,880,False,796216118331310080,en,Estados Unidos,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8023296328...,CJRubinPhoto,252,False,0.183,1379,human
2,2,2017-06-17 05:34:27,False,False,Scruffy looking nerf herder and @twitch broadc...,3307,172,594,True,875949740503859204,en,"Los Angeles, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1278890453...,SVGEGENT,1001,False,0.864,1159,human
3,3,2016-07-21 13:32:25,True,False,Wife.Godmother.Friend.Feline Fanatic! Assistan...,8433,517,633,True,756119643622735875,en,"Birmingham, AL",NaN,http://pbs.twimg.com/profile_images/1284884924...,TinkerVHELPK5,1324,False,0.889,1489,human
4,4,2012-01-15 16:32:35,False,False,Loan coach at @mancity & Aspiring DJ,88,753678,116,True,464781334,en,"England, United Kingdom",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9952566258...,JoleonLescott,4202,True,1.339,3138,human


Provided Features
created_at: timestamp
Day account was created

default_profile: Boolean
Indicating whether the account has a default profile

default_profile_image: Boolean
Indicating whether the account has a default image profile

description: String
User account description

favourites_count: Int
Total number of favourite tweets

followers_count: Int
Total number of followers

friend_count: Int
Total number of friends (people who follow back)

geo_enabled: Boolean
Indicating whether the account has the geographic location enabled

id: string
unique identifier of the account

lang: string
Language of the account

location: 
Location of the account

profile_background: string
Profile background image url

profile_image_url: String
Profile image URL

screen_name: string
username

statuses_count: int
Total number of tweets

verified: Boolean
Indicating whether the account has been verified

average_tweets_per_day: int
Average tweets posted per day (statuses_count / account_age_day)

account_age_day: int
Account age measured in days

account_type: binary
account type, bot or human

Transformed Features to consider:
Sentiment analysis for description
Number of mentions in description to other bot accounts
Length of description
Ratio Followers_count to friends count
Ratio of Followers_count to tweets per day
Ratio of friends to tweets per day
Ratio of tweets since account created
Time when account was created (Past midnight of timezone, obtained from location)
Standard deviation of avg tweets from avg tweets of bots

Total Features: 18 (provided) + 9 (created)




Structure of ML Process

Data
Data Cleaning, Imputation, etc..
Train Test split (based on account creation date)
2006-2017 used to train,test validate model
2018-2019, out of sample testing, to test how well model performs with more recent data

Features
Feature Creation
Feature Reduction (using Weight of Evidence(woe) check for feature importance, Correlation)
Run feature selection using tree base algo (random forest or smth)

Model training

Tree based models
XGB, LGBM, RF, ETC, DT

Neural Networks
NN, autoencoders(?), GNN

Get Shap for model explanability both local and global
Tune model on best model selected (either on precision/f1/recall and feature blend)

Test model on out of sample set
Get metrics(acc,precision,f1, etc..) from out of sample set

Dashboard(?) --enhancement
Transfer results to a dashboard

In [7]:
missing_values = df.isnull().sum()
missing_values

Unnamed: 0                         0
created_at                         0
default_profile                    0
default_profile_image              0
description                     7257
favourites_count                   0
followers_count                    0
friends_count                      0
geo_enabled                        0
id                                 0
lang                            7957
location                           4
profile_background_image_url    4499
profile_image_url                  1
screen_name                        0
statuses_count                     0
verified                           0
average_tweets_per_day             0
account_age_days                   0
account_type                       0
dtype: int64

In [12]:
#Transforming Bot/Human to 1s and 0s
df['account_type'] = df['account_type'].replace({'bot': 1, 'human': 0})

y = df['account_type']  # Replace 'target' with actual column name
X = df.drop(columns=['account_type'])  # Drop target to leave only features

# Step 5: Conduct the binning process using scorecardpy's `woebin` function
bins = sc.woebin(df, y='account_type')

# Step 6: Display the binning information (optional to view WoE values per variable)
for key in bins.keys():
    print(f"Binning for {key}:")
    print(bins[key])

# Step 7: Apply WoE transformation on the original dataset
df_woe = sc.woebin_ply(df, bins)

# Step 8: Display the transformed dataset
print(df_woe.head())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scorecardpy/condition_fun.py:40: FutureWarn

[INFO] creating woe binning ...
>>> There are 4 variables have too many unique non-numberic values, which might cause the binning process slow. Please double check the following variables: 
description, location, profile_image_url, screen_name
>>> Continue the binning process?
1: yes 
2: no


In [4]:

df.shape

#Transforming features
#tagging of description, construct some sort NLA
#text analysis for description
#ratio followers_count to friends count
#ratio of followers to tweets per day
# ratio of friends to tweets per day
#length of descriptions
"""
        'created_at', 'default_profile', 'default_profile_image',
       'description', 'favourites_count', 'followers_count', 'friends_count',
       'geo_enabled', 'id', 'lang', 'location', 'profile_background_image_url',
       'profile_image_url', 'screen_name', 'statuses_count', 'verified',
       'average_tweets_per_day', 'account_age_days', 'account_type'

"""



"\n        'created_at', 'default_profile', 'default_profile_image',\n       'description', 'favourites_count', 'followers_count', 'friends_count',\n       'geo_enabled', 'id', 'lang', 'location', 'profile_background_image_url',\n       'profile_image_url', 'screen_name', 'statuses_count', 'verified',\n       'average_tweets_per_day', 'account_age_days', 'account_type'\n\n"

In [7]:
df['year'] = pd.to_datetime(df['created_at']).dt.year


In [10]:
year_distribution = df['year'].value_counts().sort_index()
year_distribution

year
2006      13
2007     354
2008    1447
2009    7598
2010    4668
2011    5550
2012    4594
2013    3013
2014    2476
2015    2140
2016    2276
2017    2445
2018     835
2019      29
Name: count, dtype: int64

In [11]:
df1 = pd.read_csv("/Users/wenwei/Documents/Sku/y4s1/bt4222/project/archive/bot_detection_data.csv")


,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [14]:
df1['year'] = pd.to_datetime(df1['Created At']).dt.year
year_distribution = df1['year'].value_counts().sort_index()
year_distribution

year
2020    14767
2021    14496
2022    14718
2023     6019
Name: count, dtype: int64

In [15]:
merged_df = pd.merge(df,df1, left_on = 'screen_name',right_on = 'Username', how = 'inner')
merged_df.head()

,Unnamed: 0,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,...,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags,year_y
0,3499,2008-03-19 22:34:49,False,False,Most companies struggle to talk about what the...,782,230942,945,False,14180365,...,Certain talk instead design Mr account tree at...,83,4,3269,True,0,Dixonchester,2022-09-30 05:34:17,full scene well technology,2022
1,12083,2008-10-20 05:03:45,False,False,"Official Twitter for Playboy Centerfold, Autho...",10,261607,541,False,16862225,...,Throw eye behavior maintain development cut le...,0,2,2959,False,0,Vincentland,2020-08-24 13:01:16,whether summer analysis create administration,2020
2,17076,2008-07-05 08:56:50,False,False,Passionné de technologie. #CTO/#COO/#coachagil...,7110,10271,6979,True,15324359,...,Mouth property center over level machine baby ...,3,0,5248,False,0,West Anne,2020-10-25 11:11:29,old,2020
3,20444,2008-08-16 07:33:40,False,False,Singer / songwriter @oliverheimach,28128,3301,2220,True,15871870,...,Especially toward worry through reach black so...,14,0,3071,True,1,Norrisfurt,2020-12-25 12:59:07,table,2020
4,21640,2007-04-10 21:55:04,False,False,Biotech nerd. Tweets are my own.,92950,2309,5001,False,4083781,...,Conference level yes amount story strategy rad...,81,4,6287,False,1,Murrayland,2021-06-05 03:00:42,trip bed it politics,2021


In [3]:
df2=pd.read_json("/Users/wenwei/Documents/Sku/y4s1/bt4222/project/cresci-rtbust-2019/cresci-rtbust-2019_tweets.json")
df2.shape

(693, 2)